### Data Processing Notebook ~ By: Samuel Escalante Gutierrez
This Jupyter Notebook demonstrates the data processing workflow using the `TransformData` class from the `transform_data.py` module and load final data into 'SuccessfulApplicants' table.


### Here is where our Notebook begin  

Appends a directory path to the Python system path, be sure replace de path for your own

In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()
work_dir = os.getenv('WORK_DIR')

sys.path.append(work_dir)


Import necessary libraries and modules

In [2]:
from db_operations.transform_data import TransformData
from sqlalchemy import  inspect
from src.models.DatabaseModels import SuccessfulApplicants
from sqlalchemy.orm import sessionmaker, declarative_base
import pandas as pd
from src.database.db_connection import get_engine


In [3]:
connection = get_engine()

BASE = declarative_base()  

Session = sessionmaker(bind=connection)
session = Session()

Conected successfully to database Workshop-1!


### Data Transformation and Merge Explanation

The following code snippet demonstrates the process of transforming data using the `TransformData` class and merging two DataFrames.

#### 1. Creating Instances of TransformData
#### 2. Applying Transformations to the Candidates DataFrame  
- Including renaming columns, inserting an 'id' column, grouping by category, and adding a 'Hired' column based on specified conditions.

**Grouping by category has this structure:**
- **Development:**
    - _'Development - CMS Backend'_
    - _'Development - CMS Frontend'_
    - _'Development - FullStack'_
    - _'Development - Frontend'_
    - _'Development - Backend'_
    - _'Game Development'_
    - _'DevOps'_
    - _'Adobe Experience Manager'_

- **Quality Assurance:**
    - _'QA Automation'_
    - _'QA Manual'_

- **Security:**
    - _'System Administration'_
    - _'Security Compliance'_
    - _'Security'_

- **Data & Analytics:**
    - _'Database Administration'_
    - _'Data Engineer'_
    - _'Business Intelligence'_
    - _'Business Analytics / Project Management'_

- **Sales and Business:**
    - _'Salesforce'_
    - _'Sales'_
    - _'Client Success'_

- **Marketing and Communication:**
    - _'Social Media Community Management'_
    - _'Mulesoft'_
    - _'Technical Writing'_

- **Design:**
    - _'Design'_

#### 3. Accessing DataFrames within Instances
#### 4. Additional Transformations and Merge
#### 5. Final DataFrame Order and Export

In [10]:
# 1. 
transformer_country = TransformData('../data/countries-continent.csv')
transformer_candidates = TransformData('../data/candidates.csv')

# 2.
transformer_candidates.rename_columns()
transformer_candidates.insertar_id()
transformer_candidates.group_by_category()
transformer_candidates.agregar_contratados()

# 3.
df_country = transformer_country.df
df_candidates = transformer_candidates.df

# 4.
df_candidates['ApplicationDate'] = pd.to_datetime(df_candidates['ApplicationDate'])

# 5.
df_merged = pd.merge(df_country, df_candidates, on="Country", how="inner")

column_order = ['id','FirstName', 'LastName', 'Email', 'ApplicationDate', 'Country', 'Continent', 'YOE', 'Seniority', 'Technology', 'CodeChallengeScore', 'TechnicalInterviewScore','CategoryOfTechnology', 'Hired']
df_merged = df_merged[column_order]

df_merged.to_csv('../data/SuccessfulApplicants.csv', sep=';', encoding='utf-8', index=False)


It first checks if the 'SuccessfulApplicants' table already exists in the database before attempting to delete and recreate it. An `if-else` conditional structure is used to handle both cases.

In [8]:
if inspect(connection).has_table('SuccessfulApplicants'):
    try:
        SuccessfulApplicants.__table__.drop(connection)
        SuccessfulApplicants.__table__.create(connection)
        print("Table created successfully.")
    except Exception as e:
        print(f"Error creating table {e}")
else:
    try:
        SuccessfulApplicants.__table__.create(connection)
        print("Table created successfully.")
    except Exception as e:
        print(f"Error creating table: {e}")

Table created successfully.


Load data from a CSV file to a table called 'SuccessfulApplicants' in the database. A `try-except-finally` block is used to handle data loading and ensure that the session is closed correctly upon completion.

In [11]:
try:   
    file = TransformData('../data/SuccessfulApplicants.csv')

    file.df.to_sql('SuccessfulApplicants', connection, if_exists='replace', index=False)
    print("Data uploaded")

except Exception as e:
    print(f"Error: {e}")

finally:
    session.close()

Data uploaded
